# Лабораторна робота No2
## Наука про дані: підготовчий етап

### Мета роботи: ознайомитися з основними кроками по роботі з даними – workflow від постановки задачі до написання пояснювальної записки, зрозуміти постановку задачі.

In [1]:
import os
import pandas as pd
import urllib.request
from datetime import datetime
import numpy as np
import shutil
print("Завантажено.")

Завантажено.


#### 1. Для кожної із адміністративних одиниць України завантажити тестові структуровані файли, що містять значення VHI-індексу. Ця процедура має бути автоматизована, параметром процедури має бути індекс (номер) області. При зберіганні файлу до його імені потрібно додати дату та час завантаження.

In [2]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

def download():
    directory = 'VHI(lab2)'

    shutil.rmtree(directory)
    os.makedirs(directory, exist_ok=True)
    
    for i in range(1, 28):
        url = 'https://www.star.nesdis.noaa.gov/smcd/emb/vci/VH/get_TS_admin.php?country=UKR&provinceID={}&year1=1982&year2=2023&type=Mean'.format(i)
        
        try:
            wp = urllib.request.urlopen(url)
            text = wp.read()
            
            now = datetime.now()
            date_time = now.strftime("%Y-%m-%d_%H-%M-%S")
            
            filename = f'vhi_id={i}_{date_time}.csv'
            
            filepath = os.path.join(directory, filename)
            
            with open(filepath, 'wb') as out:
                out.write(text)
            
            print(f"VHI адмін. одиниці з індексом {i} збережено як {filename}.")
        except Exception as e:
            print(f"ПОМИЛКА ЗАВАНТАЖЕННЯ VHI адмін. одиниці з індексом {i}: {str(e)}")
download()

VHI адмін. одиниці з індексом 1 збережено як vhi_id=1_2024-03-22_11-24-27.csv.
VHI адмін. одиниці з індексом 2 збережено як vhi_id=2_2024-03-22_11-24-28.csv.
VHI адмін. одиниці з індексом 3 збережено як vhi_id=3_2024-03-22_11-24-29.csv.
VHI адмін. одиниці з індексом 4 збережено як vhi_id=4_2024-03-22_11-24-30.csv.
VHI адмін. одиниці з індексом 5 збережено як vhi_id=5_2024-03-22_11-24-31.csv.
VHI адмін. одиниці з індексом 6 збережено як vhi_id=6_2024-03-22_11-24-32.csv.
VHI адмін. одиниці з індексом 7 збережено як vhi_id=7_2024-03-22_11-24-33.csv.
VHI адмін. одиниці з індексом 8 збережено як vhi_id=8_2024-03-22_11-24-34.csv.
VHI адмін. одиниці з індексом 9 збережено як vhi_id=9_2024-03-22_11-24-34.csv.
VHI адмін. одиниці з індексом 10 збережено як vhi_id=10_2024-03-22_11-24-35.csv.
VHI адмін. одиниці з індексом 11 збережено як vhi_id=11_2024-03-22_11-24-36.csv.
VHI адмін. одиниці з індексом 12 збережено як vhi_id=12_2024-03-22_11-24-37.csv.
VHI адмін. одиниці з індексом 13 збережено як 

#### 2. Зчитати завантажені текстові файли у фрейм. Імена стовбців фрейму мають бути змістовними та легкими для сприйняття (не повинно бути спеціалізованих символів, пробілів тощо). Ця задача має бути реалізована у вигляді окремої процедури, яка на вхід приймає шлях до директорії, в якій зберігаються файли;

In [3]:
def read(directory):
    headers = ['Year', 'Week', 'SMN', 'SMT', 'VCI', 'TCI', 'VHI', 'empty']
    files = os.listdir(directory)
    result_df = pd.DataFrame()

    for i in range(len(files)):
        filepath = os.path.join(directory, files[i])
        df = pd.read_csv(filepath, header=1, names=headers, skiprows=1)  


        df = df.dropna(subset=['VHI'])
        df = df[df['VHI'] != -1]
          
        df = df.replace({'<tt>': '', '<pre>': ''})

        df['Area_ID'] = i + 1

        if 'empty' in df.columns:
            df.drop('empty', axis=1, inplace=True)

        if not df.empty:
            result_df = pd.concat([result_df, df], ignore_index=True)

    return result_df

directory = "VHI(lab2)"
df = read(directory)
print(df)

       Year  Week    SMN     SMT    VCI    TCI    VHI  Area_ID
0      1982   2.0  0.059  255.61  16.96  85.31  51.13        1
1      1982   3.0  0.060  258.56  16.90  74.56  45.73        1
2      1982   4.0  0.060  261.17  16.45  63.52  39.98        1
3      1982   5.0  0.061  263.92  15.86  53.34  34.60        1
4      1982   6.0  0.061  266.49  14.98  47.33  31.15        1
...     ...   ...    ...     ...    ...    ...    ...      ...
57586  2023  48.0  0.075  264.56  24.74  45.22  34.97       27
57587  2023  49.0  0.063  261.96  19.89  49.59  34.74       27
57588  2023  50.0  0.054  259.66  16.73  54.24  35.49       27
57589  2023  51.0  0.051  257.45  16.68  59.15  37.92       27
57590  2023  52.0  0.050  256.18  19.30  61.67  40.49       27

[57591 rows x 8 columns]


#### 3. Реалізувати процедуру, яка змінить індекси областей, які використані на порталі NOAA.

In [4]:
replaced = {
    1: 'Вінницька', 2: 'Волинська', 3: 'Дніпропетровська', 4: 'Донецька',
    5: 'Житомирська', 6: 'Закарпатська', 7: 'Запорізька', 8: 'Івано-Франківська',
    9: 'Київська', 10: 'Кіровоградська', 11: 'Луганська', 12: 'Львівська',
    13: 'Миколаївська', 14: 'Одеська', 15: 'Полтавська', 16: 'Рівенська',
    17: 'Сумська', 18: 'Тернопільська', 19: 'Харківська', 20: 'Херсонська',
    21: 'Хмельницька', 22: 'Черкаська', 23: 'Чернівецька', 24: 'Чернігівська',
    25: 'Севастопіль' , 26: 'Київ', 27: 'Республіка Крим' }

df['Area_ID'] = df['Area_ID'].replace(replaced)
print(df)

       Year  Week    SMN     SMT    VCI    TCI    VHI          Area_ID
0      1982   2.0  0.059  255.61  16.96  85.31  51.13        Вінницька
1      1982   3.0  0.060  258.56  16.90  74.56  45.73        Вінницька
2      1982   4.0  0.060  261.17  16.45  63.52  39.98        Вінницька
3      1982   5.0  0.061  263.92  15.86  53.34  34.60        Вінницька
4      1982   6.0  0.061  266.49  14.98  47.33  31.15        Вінницька
...     ...   ...    ...     ...    ...    ...    ...              ...
57586  2023  48.0  0.075  264.56  24.74  45.22  34.97  Республіка Крим
57587  2023  49.0  0.063  261.96  19.89  49.59  34.74  Республіка Крим
57588  2023  50.0  0.054  259.66  16.73  54.24  35.49  Республіка Крим
57589  2023  51.0  0.051  257.45  16.68  59.15  37.92  Республіка Крим
57590  2023  52.0  0.050  256.18  19.30  61.67  40.49  Республіка Крим

[57591 rows x 8 columns]


#### 4. Реалізувати процедури для формування вибірок наступного виду (включаючи елементи аналізу):
#### o Ряд VHI для області за вказаний рік, пошук екстремумів (min та max);

In [5]:
def extremes(df, area, year):
    min_vhi = df[(df['Area_ID'] == area) & (df['Year'] == year) & (df['VHI'] != -1)]['VHI'].min()
    max_vhi = df[(df['Area_ID'] == area) & (df['Year'] == year) & (df['VHI'] != -1)]['VHI'].max()
    return min_vhi, max_vhi
    
wanted_area = input("Адміністративна одиниця: ")
wanted_year = input("Рік: ")
wanted_year = str(wanted_year)

min_vhi, max_vhi = extremes(df, wanted_area, wanted_year)

def vhi_list(df, area, year):
    vhi_list = df.loc[(df['Area_ID'] == area) & (df['Year'] == year), ['Week', 'VHI']]
    return vhi_list
vhi_list = vhi_list(df, wanted_area, wanted_year)

print(f"Ряд VHI для адмін. одиниці {wanted_area} у {wanted_year} році:")
print(vhi_list)
print(f"Мінімальне значення VHI для адмін. одиниці {wanted_area} у {wanted_year} році: {min_vhi}")
print(f"Максимальне значення VHI для адмін. одиниці {wanted_area} у {wanted_year} році: {max_vhi}")

Адміністративна одиниця:  Київ
Рік:  2001


Ряд VHI для адмін. одиниці Київ у 2001 році:
       Week    VHI
54282   1.0  30.91
54283   2.0  35.51
54284   3.0  42.39
54285   4.0  42.98
54286   5.0  42.85
54287   6.0  43.48
54288   7.0  43.48
54289   8.0  43.49
54290   9.0  43.64
54291  10.0  44.11
54292  11.0  44.27
54293  12.0  43.98
54294  13.0  44.98
54295  14.0  48.17
54296  15.0  51.26
54297  16.0  52.66
54298  17.0  56.87
54299  18.0  61.00
54300  19.0  64.93
54301  20.0  66.24
54302  21.0  65.07
54303  22.0  64.64
54304  23.0  63.98
54305  24.0  62.28
54306  25.0  57.67
54307  26.0  50.62
54308  27.0  43.18
54309  28.0  36.00
54310  29.0  30.11
54311  30.0  25.06
54312  31.0  22.57
54313  32.0  22.30
54314  33.0  22.68
54315  34.0  23.34
54316  35.0  24.37
54317  36.0  25.26
54318  37.0  26.70
54319  38.0  28.34
54320  39.0  32.06
54321  40.0  36.96
54322  41.0  40.93
54323  42.0  43.95
54324  43.0  44.01
54325  44.0  43.66
54326  45.0  44.42
54327  46.0  44.27
54328  47.0  44.75
54329  48.0  44.50
54330  49.0  44.96
54331

#### o Ряд VHI за вказаний діапазон років для вказаних областей; виявити роки, протягом яких екстремальні посухи торкнулися більше вказаного відсотка областей по Україні (20% областей - 5 областей з 25);

In [6]:
def vhi_list(df, areas, start_year, end_year):
    result = pd.DataFrame()
    
    for area in areas:
        area_data = pd.DataFrame() 
        
        for year in range(start_year, end_year + 1):
            vhi_list = df.loc[(df['Area_ID'] == area) & (df['Year'] == str(year)), ['Week', 'VHI']]    
            vhi_list['Year'] = year
            vhi_list['Area'] = area      
            area_data = pd.concat([area_data, vhi_list], ignore_index=True)     
        result = pd.concat([result, area_data], ignore_index=True)    
    return result

selected_areas = input("Введіть адміністративні одиниці: ").split(',')
year1 = int(input("Введіть початковий рік: "))
year2 = int(input("Введіть кінцевий рік: "))

vhi_range_result = vhi_list(df, selected_areas, year1, year2)

print(f"Ряд VHI за діапазон років {year1} - {year2} для вказаних адмін. одиниць:")
print(vhi_range_result)


def drought_years(df, threshold_percentage):
    drought_years = []
    
    df_drought = df[(df['VHI'] <= 15) & (df['VHI'] != -1)]
    
    for year in df_drought['Year'].unique():
        df_year = df_drought[df_drought['Year'] == year]
        total_areas = df[df['Year'] == year]['Area_ID'].nunique()

        for area in df_year['Area_ID'].unique():
            df_area = df_year[df_year['Area_ID'] == area]
            drought_weeks = df_area['Week'].nunique()
            percentage = (drought_weeks / 52) * 100
            
            if percentage >= threshold_percentage:
                drought_years.append(year)
                break
    
    return drought_years

threshold_percentage = float(input("Введіть пороговий відсоток території,затронутої засухами: "))
drought_years_result = drought_years(df, threshold_percentage)

print(f"Роки, коли ексримальні засухи затронули більше {threshold_percentage}% областей:")
print(drought_years_result)

Введіть адміністративні одиниці:  Київ,Донецька,Луганська
Введіть початковий рік:  2010
Введіть кінцевий рік:  2022


Ряд VHI за діапазон років 2010 - 2022 для вказаних адмін. одиниць:
      Week    VHI  Year       Area
0      1.0  41.65  2010       Київ
1      2.0  42.20  2010       Київ
2      3.0  42.91  2010       Київ
3      4.0  44.20  2010       Київ
4      5.0  45.37  2010       Київ
...    ...    ...   ...        ...
2023  48.0  47.74  2022  Луганська
2024  49.0  46.93  2022  Луганська
2025  50.0  46.14  2022  Луганська
2026  51.0  47.72  2022  Луганська
2027  52.0  49.82  2022  Луганська

[2028 rows x 4 columns]


Введіть пороговий відсоток території,затронутої засухами:  20


Роки, коли ексримальні засухи затронули більше 20.0% областей:
['2007']


#### o Аналогічно для помірних посух

In [7]:
def light_drought_years(df, threshold_percentage):
    light_drought_years = []
    df_light_drought = df[(df['VHI'] > 15) & (df['VHI'] <= 35)]

    for year in df_light_drought['Year'].unique():
        df_year = df_light_drought[df_light_drought['Year'] == year]
        total_areas = df[df['Year'] == year]['Area_ID'].nunique()
        
        for area in df_year['Area_ID'].unique():
            df_area = df_year[df_year['Area_ID'] == area]
            light_drought_weeks = df_area['Week'].nunique()
            percentage = (light_drought_weeks / 52) * 100

            if percentage >= threshold_percentage:
                light_drought_years.append(year)
                break
    
    return light_drought_years

threshold_percentage = float(input("Введіть пороговий відсоток території, затронутої помірними посухами: "))
light_drought_years_result = light_drought_years(df, threshold_percentage)

print(f"Роки, в яких помірні посухи затронули більше {threshold_percentage}% областей:")
print(light_drought_years_result)


Введіть пороговий відсоток території, затронутої помірними посухами:  15


Роки, в яких помірні посухи затронули більше 15.0% областей:
['1982', '1983', '1984', '1985', '1986', '1987', '1988', '1989', '1990', '1991', '1992', '1993', '1994', '1995', '1996', '1997', '1998', '1999', '2000', '2001', '2013', '2014', '2018', '2019', '2023', '2002', '2015', '2016', '2003', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2017', '2020', '2022']
